In [26]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [27]:
load_dotenv()
llm=ChatOpenAI()

In [28]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str
    

In [29]:
def generate_joke(state: JokeState):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [30]:
def generate_explanation(state: JokeState):
    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [31]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer = checkpointer)



In [32]:
config1 = {"configurable": {"thread_id" : "1"}}
workflow.invoke({'topic': 'pizza'}, config = config1)

{'topic': 'pizza',
 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!",
 'explanation': 'This joke plays on the idea that a pizza going to therapy is a humorous and unlikely scenario. The punchline, "Because it had too many toppings and couldn\'t handle the pressure," is a play on words. It suggests that the pizza is feeling overwhelmed or stressed out by the weight of all the toppings on it, which is why it needs therapy. The joke combines the silliness of the idea of a pizza seeking therapy with a clever pun on the concept of "toppings" and "pressure."'}

In [33]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke plays on the idea that a pizza going to therapy is a humorous and unlikely scenario. The punchline, "Because it had too many toppings and couldn\'t handle the pressure," is a play on words. It suggests that the pizza is feeling overwhelmed or stressed out by the weight of all the toppings on it, which is why it needs therapy. The joke combines the silliness of the idea of a pizza seeking therapy with a clever pun on the concept of "toppings" and "pressure."'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06e622-163c-62bc-8002-f2ab9e7f9230'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '1'}, created_at='2025-07-31T23:00:19.452178+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06e622-0

In [34]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the pasta chef break up with his girlfriend? Because she was too saucy for him!',
 'explanation': 'This joke plays on the double meaning of the word "saucy." In cooking, "saucy" can refer to a dish that has a flavorful or spicy sauce. But in this joke, it is a play on words to suggest that the girlfriend had a sassy or flirtatious attitude, hence being "too saucy" for the pasta chef. The chef\'s breaking up with her is a humorous and light-hearted way to explain that he could not handle her feistiness.'}

In [35]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke plays on the idea that a pizza going to therapy is a humorous and unlikely scenario. The punchline, "Because it had too many toppings and couldn\'t handle the pressure," is a play on words. It suggests that the pizza is feeling overwhelmed or stressed out by the weight of all the toppings on it, which is why it needs therapy. The joke combines the silliness of the idea of a pizza seeking therapy with a clever pun on the concept of "toppings" and "pressure."'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06e622-163c-62bc-8002-f2ab9e7f9230'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '1'}, created_at='2025-07-31T23:00:19.452178+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06e622-0a

In [36]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke plays on the idea that a pizza going to therapy is a humorous and unlikely scenario. The punchline, "Because it had too many toppings and couldn\'t handle the pressure," is a play on words. It suggests that the pizza is feeling overwhelmed or stressed out by the weight of all the toppings on it, which is why it needs therapy. The joke combines the silliness of the idea of a pizza seeking therapy with a clever pun on the concept of "toppings" and "pressure."'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06e622-163c-62bc-8002-f2ab9e7f9230'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '1'}, created_at='2025-07-31T23:00:19.452178+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06e622-0

### Time Travel

In [37]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06e621-fa68-69c6-8000-e509a8742423"}})

StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06e621-fa68-69c6-8000-e509a8742423'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}, 'thread_id': '1'}, created_at='2025-07-31T23:00:16.534368+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06e621-fa63-6bac-bfff-e7ae8badbc7a'}}, tasks=(PregelTask(id='60203bdc-f46e-436a-47b2-40fc58d7abee', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!"}),), interrupts=())

In [38]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06e621-fa68-69c6-8000-e509a8742423"}})

{'topic': 'pizza',
 'joke': 'Why did the pizza go to the doctor?\n\nBecause it was feeling a little crusty!',
 'explanation': 'This joke plays on the double meaning of the word "crusty". In the context of a pizza, crusty refers to the texture of the pizza\'s crust, which can become hard or crispy if left out for too long. However, crusty can also be used to describe someone who is irritable or grumpy. By saying the pizza went to the doctor because it was feeling "crusty", the joke cleverly combines both meanings for a punny punchline.'}

In [39]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor?\n\nBecause it was feeling a little crusty!', 'explanation': 'This joke plays on the double meaning of the word "crusty". In the context of a pizza, crusty refers to the texture of the pizza\'s crust, which can become hard or crispy if left out for too long. However, crusty can also be used to describe someone who is irritable or grumpy. By saying the pizza went to the doctor because it was feeling "crusty", the joke cleverly combines both meanings for a punny punchline.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06e74d-108e-6b91-8002-4d5ee98ac502'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-01T01:14:05.076980+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06e74c-fdad-6e82-8001-dcf1266a7410'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topi

### Updating State

In [40]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06e621-fa68-69c6-8000-e509a8742423", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f06e757-9705-6e9f-8001-83efcc58bb21'}}

In [41]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06e757-9705-6e9f-8001-83efcc58bb21'}}, metadata={'source': 'update', 'step': 1, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-01T01:18:47.612175+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06e621-fa68-69c6-8000-e509a8742423'}}, tasks=(PregelTask(id='1a3c786e-e905-248b-c7e6-0f6f2bd3196a', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor?\n\nBecause it was feeling a little crusty!', 'explanation': 'This joke plays on the double meaning of the word "crusty". In the context of a pizza, crusty refers to the texture of the pizza\'s crust, which can become hard or crispy if left out for too long. However, crusty ca

In [43]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06e757-9705-6e9f-8001-83efcc58bb21"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa go to school?\n\nTo become a little more well-rounded!',
 'explanation': 'This joke plays on the double meaning of the phrase "well-rounded." In one sense, it can mean to be more knowledgeable or educated, which is why the samosa went to school. However, in a literal sense, a samosa is a typically triangular-shaped snack, so by going to school, it could potentially become more physically rounded in shape. The humor comes from the clever wordplay and the unexpected punchline.'}